## 🎯 Objectif du notebook `06_predict.ipynb`

Ce notebook a pour objectif de **charger les meilleurs modèles** sélectionnés lors des étapes précédentes et de **réaliser des prédictions** sur de nouvelles données.

### ❓ Pourquoi ne teste-t-on pas plusieurs modèles ici ?

Les modèles utilisés sont ceux ayant déjà été **rigoureusement comparés et validés** dans les notebooks précédents :

- 📘 `03_model_regression.ipynb`  
  ➤ Sélection du modèle **XGBoost** comme meilleur prédicteur de prix, basé sur les métriques MAE, RMSE, et R².

- 📘 `04_model_classification.ipynb`  
  ➤ Sélection du modèle **Random Forest** pour la classification binaire (tranche de prix), avec un score **Accuracy > 96%**.

Il ne s’agit donc **pas ici de réentraîner ni de comparer d’autres modèles**, mais de :
- charger les modèles finalisés,
- appliquer les transformations nécessaires,
- produire les prédictions finales.

Cette approche garantit :
- la **cohérence avec l’ensemble du pipeline**,
- un **gain de temps** pour les prédictions réelles,
- une **traçabilité claire** des choix méthodologiques pour la soutenance.

---

In [ ]:
# 📘 06_predict.ipynb — Évaluation croisée de toutes les paires de modèles

import os
import pandas as pd
import joblib
import numpy as np
from sentence_transformers import SentenceTransformer

# 🔧 Configuration
REGRESSION_MODELS = [
    "decision_tree", "gradient_boosting", "knn_regressor",
    "linear_regression", "random_forest", "ridge", "xgboost"
]

CLASSIFICATION_MODELS = [
    "decision_tree", "knn_classifier", "logistic_regression", "random_forest"
]

MODEL_DIR_REG = "../models/regression"
MODEL_DIR_CLF = "../models/classification"
SCALER_PATH = "../models/regression/scaler_notebook.pkl"
COLUMNS_PATH = "../models/columns_used_notebook.pkl"
OUTPUT_DIR = "../data/predictions_grid"

# 🔧 Chargement des objets communs
scaler = joblib.load(SCALER_PATH)
columns = joblib.load(COLUMNS_PATH)
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# 🧱 Fonction de prétraitement
def preprocess_input(description: str, fiabilite: float) -> pd.DataFrame:
    emb = embedding_model.encode([description])
    emb_dict = {f"emb_{i}": emb[0][i] for i in range(384)}
    row = {**emb_dict, "Fiabilite": fiabilite}
    df = pd.DataFrame([row])
    df = df.reindex(columns=columns, fill_value=0)
    df[["Fiabilite"]] = scaler.transform(df[["Fiabilite"]])
    return df

# 🧪 Chargement des données sources
df_source = pd.read_csv("../data/fiverr_cleaned_dl_notebook.csv")

# 🔁 Boucle sur toutes les paires de modèles
for reg_name in REGRESSION_MODELS:
    reg_model = joblib.load(f"{MODEL_DIR_REG}/{reg_name}_notebook.pkl")

    for clf_name in CLASSIFICATION_MODELS:
        clf_model = joblib.load(f"{MODEL_DIR_CLF}/{clf_name}_notebook.pkl")

        df = df_source.copy()

        # 🔮 Prédictions
        df["Prix_prédit"] = df.apply(
            lambda row: round(np.expm1(reg_model.predict(preprocess_input(row["Description"], row["Fiabilite"]))[0]) * 10, 2),
            axis=1
        )
        df["Tranche_prévue"] = df.apply(
            lambda row: clf_model.predict(preprocess_input(row["Description"], row["Fiabilite"]))[0],
            axis=1
        )

        # 💾 Sauvegarde du fichier de sortie
        filename = f"fiverr_predicted_{reg_name}_{clf_name}_notebook.csv"
        df.to_csv(os.path.join(OUTPUT_DIR, filename), index=False)
        print(f"✅ Sauvegardé : {filename}")



In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score
import pandas as pd
import numpy as np
import os

# 📂 Répertoire des fichiers de prédiction
INPUT_DIR = "../data/predictions_grid"
results = []

# 🔁 Boucle sur les fichiers de prédiction
for filename in os.listdir(INPUT_DIR):
    if filename.endswith(".csv"):
        df = pd.read_csv(os.path.join(INPUT_DIR, filename))

        # ⚠️ Ajout de la tranche réelle si manquante
        if "Tranche_réelle" not in df.columns:
            df["Tranche_réelle"] = pd.qcut(df["Prix"], q=3, labels=["Basse", "Moyenne", "Haute"])

        # 🏷️ Extraction des noms de modèles
        reg_name, clf_name = filename.replace(".csv", "").replace("fiverr_predicted_", "").split("__")

        # 📏 Calcul des métriques
        mae = mean_absolute_error(df["Prix"], df["Prix_prédit"])
        rmse = np.sqrt(mean_squared_error(df["Prix"], df["Prix_prédit"]))
        acc = accuracy_score(df["Tranche_réelle"], df["Tranche_prévue"])

        results.append({
            "Régression": reg_name,
            "Classification": clf_name,
            "MAE": round(mae, 2),
            "RMSE": round(rmse, 2),
            "Accuracy": round(acc, 4)
        })

# 📊 Affichage des résultats
df_results = pd.DataFrame(results).sort_values(by="RMSE")
print("📈 Résultats comparatifs des paires (ordonnés par RMSE) :\n")
print(df_results.to_markdown(index=False))


In [ ]:
import pandas as pd
import joblib

# Chargement du modèle entraîné (doit être un modèle à base d’arbres)
reg_model = joblib.load("models/regression/decision_tree.pkl")
columns = joblib.load("models/columns_used.pkl")

# Importance des variables
importances = reg_model.feature_importances_
features = pd.Series(importances, index=columns)

# Affichage
print("Importance de la variable Fiabilite :", round(features["Fiabilite"], 4))
print("\nTop 10 features :")
print(features.sort_values(ascending=False).head(10))